In [1]:
%matplotlib inline
# useful additional packages 

#import math tools
import numpy as np

# We import the tools to handle general Graphs
import networkx as nx

# We import plotting tools 
import matplotlib.pyplot as plt 
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

from qiskit.optimization import QuadraticProgram
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit import IBMQ

In [ ]:
#IBMQ.enable_account(token=...)

In [3]:
def qaoa_basic(n,B,C=None, backend='simulator'):

    provider = IBMQ.get_provider(hub='ibm-q') #getting the appropriate backend from IBM
    if backend=='melbourne':
        backend=provider.get_backend('ibmq_16_melbourne')
    elif backend=='simulator':
        backend=provider.get_backend('ibmq_qasm_simulator')
    
    # create a QUBO
    qubo = QuadraticProgram()
    for i in range(0,n):
        qubo.binary_var('x{}'.format(i))

    qubo.maximize(linear=C, quadratic=B)
    #print(qubo.export_as_lp_string())
    
    qaoa_mes = QAOA(quantum_instance=backend)
    qaoa = MinimumEigenOptimizer(qaoa_mes)   # using wrapper for QAOA
    
    qaoa_result = qaoa.solve(qubo)
    
    ex_time=0
    job_id = backend.jobs()[0].job_id() #get time info for latest job
    #alternative: consider last several jobs that represent subproblem
    job = backend.retrieve_job(job_id)
    time_steps=job.time_per_step()
    RUNTIME = time_steps["COMPLETED"] - time_steps["RUNNING"]
    ex_time+=RUNTIME.total_seconds()
        
    ex_time=ex_time*1000 #time in milliseconds
    
    return qaoa_result, ex_time